### CHANGELOG  
### Done:  
Batch size 32 to 1  
Made x- and y-axis same size  
Student validation: catches if students amongst data are not identical  
Data import rewrittten, now pulling all .csv files from a given folder and initializing dataframes  
Initialized institutional data feature dataframe
Initializing clickstream feature dataframe  
Demographics  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Male / Female  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1st Gen / Non  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;White / Other  
Experiment w/ neural net depth, activation functions, loss, optimizations  
Neural net fitting 
### In Progress:  
splitting for train/test w/ week 11 and week 3/4   
Permutation Importance  
~~Stochastic Hyperparameter Optimization~~  
Feature Correlation

### To Do:   
n/a   
#### Data:  
Total Boxsand Clicks  
Total Syllabus Clicks  
Total Exam Access  
Total Solutions Access  
Total Practice Problems  
Total Calendar Access  
Total YouTube Access  
Video Score Breakdown  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Quartile Division  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Sum Quartile  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Avg var after sum  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Quartile Weight?  
Prelab + Lab?  
Total Openstax Access  

In [1]:
#Import required libraries
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import glob
import re
import time
from datetime import datetime

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.layers import Dropout
import tensorboard

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

import seaborn as sns

#Clear session and set random seeds to static
tf.keras.backend.clear_session()
tf.random.set_seed(0)
np.random.seed(0)

In [2]:
%%capture
def import_data(): 
    path = r'C:\Users\OSU_bailderr\Box\Anon Data\Undergrad Students\Derrick Bailey\NN Data\week11'
    file = glob.glob(os.path.join(path, "*.csv"))
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

    dfs = []
    for f in file:
        df = pd.read_csv(f)
        dfs.append(df)
    dfs = np.array(dfs, dtype=object)

    institutional_Data = dfs[0]

    gradebook = dfs[1]
#     print(gradebook['M2%'])
    #drop students who do have missing exam scores
    gradebook['M1%'].replace('', 0, inplace=True)
    gradebook['M2%'].replace('', 0, inplace=True)
    gradebook['Final%'].replace('', 0, inplace=True)
    
    gradebook = gradebook.dropna(axis=0, how='all')
    
    gradebook = gradebook[gradebook['M1%'] != 0]
    gradebook = gradebook[gradebook['M2%'] != 0]
    gradebook = gradebook[gradebook['Final%'] != 0]
#     display(gradebook)

    clickstream1 = dfs[2] 
    clickstream2 = dfs[3]

    headers = list(clickstream1)
    clickstream2.columns = headers
    clickstream = pd.concat([clickstream1, clickstream2], ignore_index=False)

    institutional_Data_Students = institutional_Data['ID'].unique()
    institutional_Data_Students = pd.DataFrame(institutional_Data_Students)
    institutional_Data_Students = institutional_Data_Students.dropna(axis=0, how='all')
    institutional_Data_Students.columns = ['ID']

    gradebook_Students = gradebook['ID'].unique()
    gradebook_Students = pd.DataFrame(gradebook_Students)
    gradebook_Students = gradebook_Students.dropna(axis=0, how='all')
    gradebook_Students.columns = ['ID']

    clickstream_Students = clickstream['ID'].unique()
    clickstream_Students = pd.DataFrame(clickstream_Students)
    clickstream_Students = clickstream_Students.dropna(axis=0, how='all')
    clickstream_Students.columns = ['ID']

    #delete students in clickstream and institutional if not present in gradebook
    institutional_Data_Students = institutional_Data_Students[institutional_Data_Students.ID.isin(gradebook_Students.ID)].dropna()
    clickstream_Students = clickstream_Students[clickstream_Students.ID.isin(gradebook_Students.ID)].dropna()

    #check that students in institutional_Data match students in clickstream
    class StudentError(Exception):
        pass
    try:
        student_Check = clickstream_Students.iloc[:,0].str.strip().str.lower().unique().all() == institutional_Data_Students.iloc[:,0].str.strip().str.lower().unique().all() == gradebook_Students.iloc[:,0].str.strip().str.lower().unique().all()
        if student_Check != True:
            raise StudentError
    except StudentError:
        print("Students in Dataframes do not match!")
        assert False
    except(SyntaxError):
        print("Syntax Issue!")
        assert False

    #Collect demographics
    #assert demographic
#     demo = "Y"
#     institutional_Data = institutional_Data[institutional_Data["FIRST GEN IND"].str.contains(demo)==False]
    
#     display(institutional_Data)
    
    #Parse through Institutional Data to generate featureset
    institutional_Data_cols = list(institutional_Data.columns)
    # drop any and all columns that are all ZERO (necessary: columns of zeroes will influence weighting and prediction)
    institutional_Data = institutional_Data.loc[:, (institutional_Data !=0).any(axis=0)]
    # drop any and all columns that are all NaN (necessary step - matrix operation on NaN will result in all operations NaN)
    institutional_Data = institutional_Data.dropna(axis=1, how='all')
    # fill NaN with zeroes
    institutional_Data = institutional_Data.fillna(0)
    institutional_Data = institutional_Data[institutional_Data.ID.isin(institutional_Data_Students.ID)]
    institutional_Data_Features = ["ID","OVERALL OSU GPA","OSU GPA","OSU CREDITS ATTEMPT","OSU CREDITS EARNED"]
    institutional_Data = institutional_Data.loc[:, institutional_Data_Features].copy()
    
    #delete students in clickstream and gradebook if not present in institutional_data (demographic filter)
    institutional_Data_Students = institutional_Data_Students[institutional_Data_Students.ID.isin(institutional_Data.ID)].dropna()
    clickstream_Students = clickstream_Students[clickstream_Students.ID.isin(institutional_Data.ID)].dropna()
    
    institutional_Data = institutional_Data[institutional_Data.ID.isin(institutional_Data_Students.ID)].dropna()
    clickstream = clickstream[clickstream.ID.isin(clickstream_Students.ID)].dropna()
    
    #recatch students
    institutional_Data_Students = institutional_Data['ID'].unique()
    institutional_Data_Students = pd.DataFrame(institutional_Data_Students)
    institutional_Data_Students = institutional_Data_Students.dropna(axis=0, how='all')
    institutional_Data_Students.columns = ['ID']

    gradebook_Students = gradebook['ID'].unique()
    gradebook_Students = pd.DataFrame(gradebook_Students)
    gradebook_Students = gradebook_Students.dropna(axis=0, how='all')
    gradebook_Students.columns = ['ID']

    clickstream_Students = clickstream['ID'].unique()
    clickstream_Students = pd.DataFrame(clickstream_Students)
    clickstream_Students = clickstream_Students.dropna(axis=0, how='all')
    clickstream_Students.columns = ['ID']
    
    #Parse through Gradebook Data to generate featureset
    gradebook_cols = list(gradebook.columns)
    #feature list for gradebook
    gradebook_features = ['ID', 'raw','online','HW%','RW%','Rec %','Lab %','M1%','M2%','Final%','Overall Grade %']
    gradebook = gradebook[gradebook_features]
    del gradebook['M1%']
    del gradebook['M2%']
    del gradebook['Final%']
    gradebook = gradebook[gradebook.ID.isin(institutional_Data.ID)].dropna()
    final_Grade = gradebook['Overall Grade %']
    final_Grade = pd.DataFrame([final_Grade]).T
    final_Grade = final_Grade.values
    final_Grade = pd.DataFrame(final_Grade)
    # remove final grade from gradebook
    del gradebook['Overall Grade %']
    # drop any and all columns that are all ZERO (necessary: columns of zeroes will influence weighting and prediction)
    gradebook = gradebook.loc[:, (gradebook !=0).any(axis=0)]
    # drop any and all columns that are all NaN (necessary step - matrix operation on NaN will result in all operations NaN)
    gradebook = gradebook.dropna(axis=1, how='all')
    # fill NaN with zeroes
    gradebook = gradebook.fillna(0)
    # recatch cols var
    cols = list(gradebook.columns)
    gradebook = gradebook[gradebook.ID.isin(gradebook_Students.ID)]

    #Parse through Clickstream Data to generate featureset
    clickstream_cols = list(clickstream.columns)
    clickstream.set_index=clickstream['ID']
    # drop any and all columns that are all ZERO (necessary: columns of zeroes will influence weighting and prediction)
    clickstream = clickstream.loc[:, (clickstream !=0).any(axis=0)]
    # drop any and all columns that are all NaN (necessary step - matrix operation on NaN will result in all operations NaN)
    clickstream = clickstream.dropna(axis=1, how='all')
    # fill NaN with zeroes
    clickstream = clickstream.fillna(0)
    clickstream = clickstream[clickstream.ID.isin(clickstream_Students.ID)]

    # now to parse through for specific features
    # create list out of student column
    student_list = clickstream[['ID']]
    student_list_unique = pd.DataFrame(student_list['ID'].unique())
    student_list_unique.columns = ['ID']
    clickstream_features = ['m1_sols', 'm2_sols','f_sols', '#Practice', '#Fundamental', 'Calendar', 'KALTURA', 'youtube']
    clickstream_cut = clickstream[clickstream['Page title'].str.contains("|".join(clickstream_features))]
    
    studentdf = []
    studentdf = [x for _, x in clickstream_cut.groupby(by='ID')]
    
    clickstream_Data = []
    clickstream_Data = pd.DataFrame(columns = clickstream_features)
    clickstream_Data = student_list_unique.append(clickstream_Data)

    count = []

    #automated feature-by-feature approach
    for feature in clickstream_features:
        for pos in studentdf:
            for ID in pos['ID'].unique():
                ID = pos['ID'].unique()
                count.append(pos['Page title'].str.contains(feature).sum())
        clickstream_Data[feature] = count
        count.clear()

    frames = pd.merge(institutional_Data, gradebook, on='ID')
    frames = pd.merge(frames, clickstream_Data, on='ID')

#     final_Grade = final_Grade[final_Grade.ID.isin(institutional_Data_Students.ID)].dropna()
    
    input_Features = frames
    feature_num = len(input_Features.columns)
    # display(input_Features)
    return input_Features, final_Grade, studentdf, student_list_unique


In [3]:
%%capture
def debug(debug=False):
    while debug == True:
        debug_input = int(input("Invoke the rite of passage! \n \
        1: Final Grades Dataframe \n \
        2: Student Dataframe \n \
        3: Input Features Dataframe \n \
        4: Exit \n"))
        if debug_input == 1:
            print("Final Grades Dataframe")
            display(final_Grade)
            break
        elif debug_input == 2:
            print("Student Dataframe")
            display(studentdf)
            break
        elif debug_input == 3:
            print("Input Features Dataframe")
            display(input_Features)
            break
        elif debug_input == 4:
            debug_input = False
            break
        else:
            print("Invalid input")

In [4]:
%%capture
def process_data(input_Features, final_Grade):
    # Split data for train/test
    x_train, x_test, y_train, y_test = train_test_split(input_Features, final_Grade, 
                                                        test_size=0.33, random_state=0)
    # test
    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    # delete student column before normalization but saving student column
    x_train_student = x_train[:,0]
    x_test_student = x_test[:,0]
    x_train = np.delete(x_train, 0, 1)
    x_test = np.delete(x_test, 0, 1)
    
    # Normalization
    sc = MinMaxScaler(feature_range = (0,1))
    x_train_norm = sc.fit_transform(x_train)
    x_test_norm = sc.fit_transform(x_test)
    y_train_norm = sc.fit_transform(y_train)
    y_test_norm = sc.fit_transform(y_test)

    x_train_norm = np.reshape(x_train_norm, (x_train_norm.shape[0], x_train_norm.shape[1], 1))
    x_test_norm = np.reshape(x_test_norm, (x_test_norm.shape[0], x_test_norm.shape[1], 1))
    
    return x_train_norm, x_test_norm, y_train_norm, y_test_norm, x_train_student, x_test_student, sc

In [5]:
%%capture
def student_cross_ref(student_list_unique, final_Grade, predicted_Grade):
    # We must cross-reference the split data with the final grade master list
    # and pull only the matching student numbers from the final grade master list.

    # Start by zipping together student dataframe and final grade dataframe.
    final_Grade = np.hstack((student_list_unique, final_Grade))
    final_Grade = pd.DataFrame(final_Grade)
    # To make our lives easy, let's name the columns for our predicted and final grade dataframes.
    final_Grade.columns = ['FStudent','FGrade']
    predicted_Grade.columns = ['PStudent','PGrade']
    # Now, we compare the student columns for these dataframes.
    merged_Grades = final_Grade.merge(predicted_Grade, left_on = 'FStudent', right_on = 'PStudent', how = 'left')
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     display(merged_Grades)
    merged_Grades = merged_Grades[merged_Grades['PStudent'].notna()]
    return merged_Grades

In [6]:
def hyperparameter_optimization(hyp_opt):   
    if hyp_opt == "Y":
        param_list = [[*range(100,1100,100)], 
                      [*map(lambda x: 2 ** x, range(8,-1,-1))], 
                      [*map(lambda x: 1*10**(-x), range(5))], 
#                       [*map(lambda x: 1*10**(-1-x), range(5))], 
                      list(np.linspace(0.05,0.3,6).round(decimals=2)),
                      list(np.linspace(0.05,0.3,6).round(decimals=2)), 
                      list(np.linspace(0.05,0.3,6).round(decimals=2)), 
                      list(np.linspace(0.05,0.3,6).round(decimals=2)), 
                      list(np.linspace(0.05,0.3,6).round(decimals=2)), 
                      list(np.linspace(0.05,0.3,6).round(decimals=2)),
                      [*map(lambda x: 2 ** x, range(10))],
                      [*map(lambda x: 2 ** x, range(10))], 
                      [*map(lambda x: 2 ** x, range(10))]]
        hype_var_list_index = ['num_epochs', 'batch_size', 'lr', 'recurrent_dropout1', 'recurrent_dropout2', 'recurrent_dropout3', 'dropout1', 'dropout2', 'dropout3', 'gru_units1', 'gru_units2', 'gru_units3']

        num_epochs, batch_size, lr, recurrent_dropout1, recurrent_dropout2, recurrent_dropout3, dropout1, dropout2, dropout3, gru_units1, gru_units2, gru_units3 = [param_list[i] for i in range(len(param_list))]
        hype_var_list = pd.DataFrame(index=hype_var_list_index,
                                      data=([num_epochs], 
                                      [batch_size], 
                                      [lr], 
                                      [recurrent_dropout1], 
                                      [recurrent_dropout2], 
                                      [recurrent_dropout3], 
                                      [dropout1], 
                                      [dropout2], 
                                      [dropout3], 
                                      [gru_units1], 
                                      [gru_units2], 
                                      [gru_units3]))
        optimize_gate = True
#         hype_var_list.DataFrame()
#         print(hype_var_list)
#         print(hype_var_list)
        return hype_var_list, optimize_gate
    if hyp_opt == "N":
        param_list = [500, 1, 1e-4, 0.15, 0.15, 0.15, 0.2, 0.2, 0.2, 3125, 625, 125]
        num_epochs, batch_size, lr, recurrent_dropout1, recurrent_dropout2, recurrent_dropout3, dropout1, dropout2, dropout3, gru_units1, gru_units2, gru_units3 = [param_list[i] for i in range(len(param_list))]
        hype_var_list = [num_epochs, 
                         batch_size, 
                         lr, 
                         recurrent_dropout1, 
                         recurrent_dropout2, 
                         recurrent_dropout3, 
                         dropout1, 
                         dropout2, 
                         dropout3, 
                         gru_units1, 
                         gru_units2, 
                         gru_units3]
        optimize_gate = False
        return hype_var_list, optimize_gate
    if hyp_opt != "Y" and hyp_opt != "N":
        print("Try Again!")
        hyp_opt = hyperparameter_optimization(input("Optimize? "))

In [7]:
def EvaluateModel(hype_var_list, optimize_gate, x_train_norm, y_train_norm, x_test_norm, x_test_student, sc, student_list_unique, final_Grade):
    if optimize_gate == True:
        index = hype_var_list.index
        for i in hype_var_list:
            num_epochs, batch_size, lr, recurrent_dropout1, recurrent_dropout2, recurrent_dropout3, dropout1, dropout2, dropout3, gru_units1, gru_units2, gru_units3 = hype_var_list[i]
        for a in num_epochs:
#             print("num_epochs: ", a, "\n")
            for b in batch_size:
#                 print("batch_size: ", b, "\n")
                for c in lr:
#                     print("lr: ", c, "\n")
#                         print("decay: ", d, "\n")
                        for d in recurrent_dropout1:
#                             print("recurrent_dropout1: ", e, "\n")
                            for e in recurrent_dropout2:
#                                 print("recurrent_dropout2: ", f, "\n")
                                for f in recurrent_dropout3:
#                                     print("recurrent_dropout3: ", g, "\n")
                                    for g in dropout1:
#                                         print("dropout1: ", h, "\n")
                                        for h in dropout2:
#                                             print("dropout2: ", i, "\n")
                                            for i in dropout3:
#                                                 print("dropout3: ", j, "\n")
                                                for j in gru_units1:
#                                                     print("gru_units1: ", k, "\n")
                                                    for k in gru_units2:
#                                                         print("gru_units2: ", l, "\n")
                                                        for l in gru_units3:
#                                                             print("gru_units3: ", m, "\n")
                                                            hype_list = [a,b,c,d,e,f,g,h,i,j,k,l]
                                                            BuildModel(x_train_norm, y_train_norm, x_test_norm, x_test_student, sc, hype_list, student_list_unique, final_Grade)
        #BuildModel()
#         pass
    if optimize_gate == False:
        #train on defaults
#         print(hype_var_list)
#         for i in hype_var_list:
        num_epochs, batch_size, lr, recurrent_dropout1, recurrent_dropout2, recurrent_dropout3, dropout1, dropout2, dropout3, gru_units1, gru_units2, gru_units3 = hype_var_list
#         print(num_epochs, batch_size, lr, recurrent_dropout1, recurrent_dropout2, recurrent_dropout3, dropout1, dropout2, dropout3, gru_units1, gru_units2, gru_units3)
        hype_list = [num_epochs, batch_size, lr, recurrent_dropout1, recurrent_dropout2, recurrent_dropout3, dropout1, dropout2, dropout3, gru_units1, gru_units2, gru_units3]
        BuildModel(x_train_norm, y_train_norm, x_test_norm, x_test_student, sc, hype_list, student_list_unique, final_Grade)
#         pass

In [8]:
def visualize_chaos(merged_Grades, loss_df, mae, mse, fit):
    # Visualize the chaos
    fig, (ax1,ax2) = plt.subplots(1,2)
    ax1.scatter(merged_Grades.iloc[:,3],merged_Grades.iloc[:,1], c=merged_Grades.iloc[:,3], cmap = 'jet_r', edgecolors='black')
    ax1.set_xlabel('Predicted Grade')
    ax1.set_ylabel('Actual Grade')
    min_predicted = merged_Grades['PGrade'].min()
    ax1.set_xlim(0,100)
    ax1.set_ylim(0,100)

    ax2.plot(loss_df[['loss', 'val_loss']])
#     print(loss_df)
    ax2.set_xlabel("Number of Epochs")
    ax2.set_ylabel("Loss")
    ax2.plot(fit.history['loss'], label = 'train')
    ax2.plot(fit.history['val_loss'], label = 'test')
    ax2.legend()
    textstr = '\n'.join((
        r'$MAE = %1.3f$' %mae,
        r'$MSE = %1.3f$' %mse))

    ax1.text(0.05, 0.95, textstr, transform = ax1.transAxes, fontsize = 14, va = 'top')

    lims1 = [
        np.min([ax1.get_xlim(), ax1.get_ylim()]),  # min of both axes
        np.max([ax1.get_xlim(), ax1.get_ylim()]),  # max of both axes
    ]

    lims2 = [
         np.min([ax2.get_xlim(), ax2.get_ylim()]),  # min of both axes
         np.max([ax2.get_xlim(), ax2.get_ylim()]),  # max of both axes
    ]

    # now plot both limits against eachother
    ax1.plot(lims1, lims1, 'k-', alpha=0.5, zorder=0)

    ax1.grid()
    fig.tight_layout()
    fig.subplots_adjust(right=2, wspace=0.2)
    
#     datetime = time.strftime("%Y-%m-%d %H:%M:%S")
#     fig.savefig("Figure "+ datetime.str() + ".png")
    
    plt.show()
    figname = input('Graph name: ')
    fig.savefig("Graphs/{}.svg".format(figname), transparent=False, facecolor = 'w', bbox_inches='tight')
    fig.savefig("Graphs/{}.png".format(figname), transparent=False, facecolor = 'w', bbox_inches='tight')

In [9]:
def correlation_heatmap(train):
    train = pd.DataFrame(train, columns=cols)
    correlations = train.corr()

    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, vmax=1.0, vmin=-1.0, center=0, fmt='.2f',
                square=True, linewidths=1, annot=True, cmap = 'vlag', cbar_kws={"shrink": .70})
    savegraph = input('Do you want to save this graph? Y/N: ')
    if savegraph == "Y":
        figname = input('Graph name: ')
        fig.savefig("Graphs/{}.svg".format(figname), transparent=False, facecolor = 'w', dpi=150)
        fig.savefig("Graphs/{}.png".format(figname), transparent=False, facecolor = 'w', dpi=150)
    plt.show();
# input_Features, final_Grade, studentdf, student_list_unique = import_data()
# cols = list(input_Features.keys())
# cols.pop(0)
# x_train_norm, x_test_norm, y_train_norm, y_test_norm, x_train_student, x_test_student, sc = process_data(input_Features, final_Grade)
# x_train_norm = pd.DataFrame(x_train_norm, columns=cols)
# y_train_norm = pd.DataFrame(y_train_norm, columns=['final_Grade'])
# combined = pd.concat([x_train_norm, y_train_norm], axis=1)
# cols.append('final_Grade')
# correlation_heatmap(combined)

In [10]:
### Permution Importance


In [11]:
# savegraph = input('Do you want to save this graph? Y/N: ')
# if savegraph == "Y":
#     figname = input('Graph name: ')
#     fig.savefig("Graphs/{}.svg".format(figname), transparent=False, facecolor = 'w', dpi=150)
#     fig.savefig("Graphs/{}.png".format(figname), transparent=False, facecolor = 'w', dpi=150)

In [12]:
# display_Students = input("Display students? Y/N: ")
# display_Students.upper()
# if display_Students == "Y":
#     with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#         display(merged_Grades)

In [13]:
def BuildModel(x_train_norm, y_train_norm, x_test_norm, x_test_student, sc, hype_list, student_list_unique, final_Grade):
    num_epochs, batch_size, lr, recurrent_dropout1, recurrent_dropout2, recurrent_dropout3, dropout1, dropout2, dropout3, gru_units1, gru_units2, gru_units3 = hype_list
#     print(num_epochs, batch_size, lr, recurrent_dropout1, recurrent_dropout2, recurrent_dropout3, dropout1, dropout2, dropout3, gru_units1, gru_units2, gru_units3)
    print(x_train_norm.shape)
    #GRU model
#     type(x_train_norm)
    tf.keras.backend.set_floatx('float64')
    with tf.device('/cpu:0'):
        # NN Parameters
        # Initialization
        RNN = Sequential()
        start = time.perf_counter()
        # 1st GRU layer & Dropout regularisation
        RNN.add(GRU(units=gru_units1, return_sequences=True, recurrent_dropout=recurrent_dropout1, input_shape=(x_train_norm.shape[1], 1)))
        RNN.add(Dropout(dropout1))
#         RNN.add(BatchNormalization())
        # 2nd GRU Layer & Dropout regularisation
        RNN.add(GRU(units=gru_units2, return_sequences=True, recurrent_dropout=recurrent_dropout2))
        RNN.add(Dropout(dropout2))
#         RNN.add(BatchNormalization())
        # 3rd GRU Layer & Dropout regularisation
        RNN.add(GRU(units=gru_units3, recurrent_dropout=recurrent_dropout3))
        RNN.add(Dropout(dropout3))
#         RNN.add(BatchNormalization())
        # 4th GRU Layer
#         RNN.add(GRU(units=50, recurrent_dropout=0.1))
    #   RNN.add(BatchNormalization())
        # Output Layer
        RNN.add(Dense(units = 1))
        # Compiling RNN
        loss = tf.keras.losses.MeanSquaredError()
        decayed_lr = tf.keras.optimizers.schedules.ExponentialDecay(lr, 1000, 0.96, staircase=False)
        opt = tf.keras.optimizers.Adam(learning_rate=decayed_lr)
        RNN.compile(optimizer=opt, loss=loss)

        #Early Stop Callback
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
        min_delta = 5e-6, 
        patience=30, 
        restore_best_weights = True)
        #Checkpoint Callback
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=r'C:\Users\OSU_bailderr\Box\Anon Data\Undergrad Students\Derrick Bailey\NN Repository\Checkpoints',
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
        #Tensorboard Callback
        logdir='logs\\fit\\' + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard = keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)

        # Fitting RNN to training set
        fit = RNN.fit(x_train_norm, y_train_norm, epochs = num_epochs, batch_size = batch_size, validation_split=0.15, validation_data=None, callbacks=[early_stop, tensorboard], shuffle=True)
        loss_df = pd.DataFrame(fit.history)
        print(RNN.summary())
    elapsed = time.perf_counter() - start
    print('Elapsed %.3f seconds.' % elapsed)
    # Evaluate mode
    # history = RNN.evaluate(x_test_norm, y_test_norm, batch_size = 1)

    # Predicted Grade
    predicted_Grade = RNN.predict(x_test_norm)
    # predicted_Grade_Dataset = np.zeros(shape=(len(predicted_Grade), 10))
    # predicted_Grade_Dataset[:,0] = predicted_Grade[:,0]
    predicted_Grade = sc.inverse_transform(predicted_Grade)[:,0]
    predicted_Grade = np.vstack((x_test_student, predicted_Grade))
    predicted_Grade = predicted_Grade.T
    predicted_Grade = pd.DataFrame(predicted_Grade)
    
    
#     return predicted_Grade, loss_df

    merged_Grades = student_cross_ref(student_list_unique, final_Grade, predicted_Grade)
    #compute MAE & MSE
    mae = tf.keras.losses.MeanAbsoluteError()
    mse = tf.keras.losses.MeanSquaredError()
    #write MAE from scratch, compare to tf???
    mae = mae(merged_Grades.iloc[:,3],merged_Grades.iloc[:,1]).numpy()
    mse = mse(merged_Grades.iloc[:,3],merged_Grades.iloc[:,1]).numpy()
    visualize_chaos(merged_Grades, loss_df, mae, mse, fit)

In [14]:
def main():
    input_Features, final_Grade, studentdf, student_list_unique = import_data()
    x_train_norm, x_test_norm, y_train_norm, y_test_norm, x_train_student, x_test_student, sc = process_data(input_Features, final_Grade)
    hype_var_list, optimize_gate = hyperparameter_optimization(hyp_opt = input("Optimize Hyperparameters? Y/N: "))
    EvaluateModel(hype_var_list, optimize_gate, x_train_norm, y_train_norm, x_test_norm, x_test_student, sc, student_list_unique, final_Grade)
#     predicted_Grade, loss_df = NeuralNetwork(x_train_norm, y_train_norm, x_test_norm, x_test_student, sc)
    
main()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\OSU_bailderr\\Box\\Anon Data\\Undergrad Students\\Derrick Bailey\\NN Data\\week11'

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs